In [ ]:
!pip install -q datasets evaluate gradio transformers seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.0 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import evaluate
import gradio as gr
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer
)

# ✅ Load dataset
df = pd.read_csv("legal_ner_dataset_entity_per_row.csv")
print("✅ Dataset Loaded")

# ✅ Tokenizer and labels
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
unique_labels = sorted(df["label"].dropna().unique())
label_list = ["O"] + [f"B-{l}" for l in unique_labels] + [f"I-{l}" for l in unique_labels]
label_to_id = {l: i for i, l in enumerate(label_list)}
id_to_label = {i: l for l, i in label_to_id.items()}

# ✅ Label aligner
def align_labels_with_tokens(sentence, entities):
    encoding = tokenizer(sentence, return_offsets_mapping=True, truncation=True, add_special_tokens=False)
    labels = ["O"] * len(encoding["offset_mapping"])

    for ent_text, ent_type in entities:
        start = sentence.find(ent_text)
        if start == -1:
            continue
        end = start + len(ent_text)
        for i, (token_start, token_end) in enumerate(encoding["offset_mapping"]):
            if token_start >= start and token_end <= end:
                if token_start == start:
                    labels[i] = f"B-{ent_type}"
                else:
                    labels[i] = f"I-{ent_type}"

    tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])
    label_ids = [label_to_id[l] for l in labels]
    return tokens, label_ids

formatted = []
for sentence in df["sentence"].unique():
    entities = df[df["sentence"] == sentence][["entity", "label"]].values.tolist()
    tokens, tags = align_labels_with_tokens(sentence, entities)
    formatted.append({"tokens": tokens, "ner_tags": tags})

dataset = Dataset.from_list(formatted).train_test_split(test_size=0.3)

# ✅ Tokenization
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=64
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

# ✅ Load model
model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(label_list))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Trainer setup
args = TrainingArguments(
    output_dir="legal-ner-bert",
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="no",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.03,
    report_to="none"


)

metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_labels = [[id_to_label[l] for l in label if l != -100] for label in labels]
    true_preds = [[id_to_label[p] for (p, l) in zip(pred, label) if l != -100]
                  for pred, label in zip(predictions, labels)]
    results = metric.compute(predictions=true_preds, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"].remove_columns(["tokens", "ner_tags"]),
    eval_dataset=tokenized_dataset["test"].remove_columns(["tokens", "ner_tags"]),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# ✅ Train model
trainer.train()

# ✅ Inference function with CLS/SEP removed
def ner_infer(text):
    try:
        # Get offset mapping separately (not passed to model)
        encoding = tokenizer(text, return_offsets_mapping=True, truncation=True)
        offsets = encoding["offset_mapping"]

        # Get model inputs
        inputs = tokenizer(text, return_tensors="pt", truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Predict
        with torch.no_grad():
            logits = model(**inputs).logits

        preds = torch.argmax(logits, dim=2)[0].cpu().numpy()
        input_ids = inputs["input_ids"][0]
        tokens = tokenizer.convert_ids_to_tokens(input_ids)

        result = []
        current_entity = ""
        current_label = ""

        for token, pred in zip(tokens, preds):
            if token in ["[CLS]", "[SEP]"]:
                continue  # skip special tokens

            label = id_to_label[pred]
            word = token.replace("##", "")

            if label == "O":
                if current_entity:
                    result.append(f"[{current_entity.strip()} → {current_label}]")
                    current_entity = ""
                    current_label = ""
                result.append(word)
            elif label.startswith("B-"):
                if current_entity:
                    result.append(f"[{current_entity.strip()} → {current_label}]")
                current_entity = word + " "
                current_label = label[2:]
            elif label.startswith("I-") and current_label == label[2:]:
                current_entity += word + " "
            else:
                if current_entity:
                    result.append(f"[{current_entity.strip()} → {current_label}]")
                result.append(word)
                current_entity = ""
                current_label = ""

        if current_entity:
            result.append(f"[{current_entity.strip()} → {current_label}]")

        return " ".join(result)

    except Exception as e:
        return f"Error: {str(e)}"


# ✅ Gradio UI
gr.Interface(
    fn=ner_infer,
    inputs=gr.Textbox(lines=4, placeholder="Enter a legal sentence here..."),
    outputs="text",
    title="📜 Legal Entity Extractor (BERT-Base-Cased)",
    description="Extracts PERSON, ORG, DATE, LAW from legal text using fine-tuned BERT."
).launch()


✅ Dataset Loaded


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/tmp/ipython-input-3705342335.py:122: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.534600,0.004686,1.000000,1.000000,1.000000,1.000000
2,0.004900,0.001201,1.000000,1.000000,1.000000,1.000000
3,0.002700,0.000996,1.000000,1.000000,1.000000,1.000000


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6760f4ca7663fab81e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


John Smith from Lex & Co. signed the agreement on 12th March 2021 under the Corporate Law.

Mary Johnson, representing LegalBridge LLP, approved the contract on 23rd August 2020 as per the Privacy Regulation.

The deal between Alan Brown and Apex Law Firm was finalized on 9th January 2019 following the Cybersecurity Act.

Emily Davis of Nova Legal agreed on 30th November 2023 under the Civil Procedure Code.

Michael Clark signed with Hudson & Hudson on 17th June 2020 pursuant to the Arbitration Act.

Olivia Martin, a partner at Orion Law Group, accepted the terms on 5th February 2022 under the Employment Standards Act.

The partnership between Sophia Wilson and Quantum Legal Partners began on 1st May 2018 in accordance with the Contract Law.

Benjamin Lee concluded negotiations with Vertex Legal on 14th April 2021 referencing the Financial Services Act.

Chloe Taylor of Horizon Legal Advisors executed the agreement on 6th October 2022 under the Intellectual Property Act.

Ethan Harris, from Sentinel Law Chambers, signed the settlement on 21st December 2023 under the Digital Rights Law.